In [ ]:
!pip install google-cloud-bigquery --quiet

In [ ]:
from google.cloud import bigquery

# Set the project ID explicitly if the client doesn't pick it up automatically
PROJECT_ID = 'stgw-shared-ai-dev'

# The BigQuery client should automatically use the Service Account
# associated with your Colab Enterprise notebook/runtime.
client = bigquery.Client(project=PROJECT_ID)

print(f"BigQuery Client initialized for project: {PROJECT_ID}")
# You can now proceed to run your BigQuery queries (CREATE TABLE, SELECT, etc.)

In [ ]:
from google.cloud import bigquery

client = bigquery.Client()
table_id = "stgw-shared-ai-dev.parquet_data_set.IMP_profile_joined_behavioral_base"

# Query BigQuery's information schema to get column names
query_columns = f"""
SELECT column_name
FROM `{table_id.split('.')[0]}`.{table_id.split('.')[1]}.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = '{table_id.split('.')[-1]}'
"""

columns = [row['column_name'] for row in client.query(query_columns).result()]

print (f"Fetched {len(columns)} column names successfully.")

# Missing Percentage calculation

In [ ]:
from google.cloud import bigquery

# Initialize client
client = bigquery.Client(project="stgw-shared-ai-dev")

# --- PARAMETERS ---
dataset = "parquet_data_set"
table = "IMP_profile_joined_behavioral_base"
dataset_ref = f"{client.project}.{dataset}"

# --- STEP 1: Get column metadata ---
query_columns = f"""
SELECT column_name, data_type, ordinal_position
FROM `{dataset_ref}.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = '{table}'
ORDER BY ordinal_position
"""
columns = client.query(query_columns).to_dataframe()

# --- STEP 2: Build COUNTIF expressions ---
countif_clauses = []
for _, row in columns.iterrows():
    col, dtype = row["column_name"], row["data_type"]
    if dtype == "STRING":
        expr = f"COUNTIF({col} IS NULL OR TRIM({col}) = '') AS missing_{col}"
    else:
        expr = f"COUNTIF({col} IS NULL) AS missing_{col}"
    countif_clauses.append(expr)

countif_clause_str = ",\n        ".join(countif_clauses)

# --- STEP 3: Build union (unpivot) statements ---
union_clauses = []
for _, row in columns.iterrows():
    col, dtype = row["column_name"], row["data_type"]
    union_clauses.append(
        f"SELECT '{col}' AS column_name, '{dtype}' AS data_type, "
        f"(missing_{col} * 100.0 / total_rows) AS missing_percentage FROM MissingnessData"
    )
union_clause_str = "\nUNION ALL\n".join(union_clauses)

# --- STEP 4: Assemble final query ---
final_query = f"""
WITH MissingnessData AS (
    SELECT
        COUNT(*) AS total_rows,
        {countif_clause_str}
    FROM `{dataset_ref}.{table}`
)
{union_clause_str}
ORDER BY missing_percentage DESC
"""

print("✅ Query built successfully!\n")

# --- STEP 5: Run the query ---
df_result = client.query(final_query).to_dataframe()
df_result.head(20)  # Display top 20 columns with missingness


In [ ]:
# dataframe: df_result
# uuid: A4B49738-F56C-4CE8-8BE1-8FD427D51CAE
# output_variable:

import google.colabsqlviz.explore_dataframe as _vizcell
_vizcell.explore_dataframe(df_or_df_name='df_result', uuid='A4B49738-F56C-4CE8-8BE1-8FD427D51CAE')

# Column Profiling

🧩 What the script does

It queries your BigQuery table column by column (one at a time).

For each column, it computes:

total number of rows

number of unique values

% of unique values (relative to total rows)

top 3 most frequent values with counts (top_values)

an imbalance ratio (share of the most frequent value)

It then appends each column’s profile into one combined DataFrame →
✅ 227 rows total (one per column)

In [ ]:
!pip install google-cloud-bigquery --quiet
from google.cloud import bigquery
import pandas as pd
import re

client = bigquery.Client(project="stgw-shared-ai-dev")

dataset = "parquet_data_set"
table = "IMP_profile_joined_behavioral_base"
dataset_ref = f"{client.project}.{dataset}"

# --- 1️⃣ Get column list ---
query_columns = f"""
SELECT column_name, data_type
FROM `{dataset_ref}.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = '{table}'
ORDER BY ordinal_position
"""
cols = client.query(query_columns).to_dataframe()

# --- 2️⃣ Define helper to quote column safely ---
def safe(col):
    return f"`{col.replace('`','')}`"

# --- 3️⃣ Build and run per-column query safely ---
profiles = []
for _, row in cols.iterrows():
    col, dtype = row["column_name"], row["data_type"]
    cq = safe(col)

    # build small subquery
    q = f"""
    SELECT
      '{col}' AS column_name,
      '{dtype}' AS data_type,
      COUNT(*) AS total_rows,
      COUNT(DISTINCT {cq}) AS unique_values,
      ROUND(COUNT(DISTINCT {cq}) * 100.0 / COUNT(*), 2) AS uniqueness_pct,
      (
        SELECT ARRAY_AGG(STRUCT(val, freq) ORDER BY freq DESC LIMIT 3)
        FROM (
          SELECT {cq} AS val, COUNT(*) AS freq
          FROM `{dataset_ref}.{table}`
          WHERE {cq} IS NOT NULL
          GROUP BY {cq}
          ORDER BY freq DESC
          LIMIT 3
        )
      ) AS top_values,
      (
        SELECT MAX(freq) / SUM(freq)
        FROM (
          SELECT {cq} AS val, COUNT(*) AS freq
          FROM `{dataset_ref}.{table}`
          WHERE {cq} IS NOT NULL
          GROUP BY {cq}
        )
      ) AS imbalance_ratio
    FROM `{dataset_ref}.{table}`
    LIMIT 1
    """

    try:
        subdf = client.query(q).to_dataframe()
        profiles.append(subdf)
    except Exception as e:
        print(f"⚠️ Skipping column '{col}' due to SQL error: {e}")
        continue

# --- 4️⃣ Combine results ---
if profiles:
    df_profile = pd.concat(profiles, ignore_index=True)
else:
    raise RuntimeError("No valid column profiles generated.")

# --- 5️⃣ Recommendations ---
def rec(row):
    if row.unique_values <= 1:
        return "⚠️ Constant — drop"
    if row.unique_values < 10:
        return "✅ Good categorical"
    if row.unique_values < 50:
        return "ℹ️ Moderate — check imbalance"
    if row.unique_values < 200:
        return "🧩 Consider binning/grouping"
    return "📉 Continuous / high-cardinality"

df_profile["recommendation"] = df_profile.apply(rec, axis=1)

# --- 6️⃣ Display ---
pd.set_option("display.max_colwidth", None)
df_profile[[
    "column_name", "data_type", "unique_values",
    "uniqueness_pct", "imbalance_ratio", "recommendation", "top_values"
]].head(30)


In [ ]:
len(df_profile)


In [ ]:
missing = set(cols["column_name"]) - set(df_profile["column_name"])
missing


In [ ]:
# --- STEP 7️⃣: Save and download results in Colab ---

from google.colab import files
import datetime

# Create a timestamped filename (optional but helps versioning)
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"column_profiling_results_{timestamp}.csv"

# Save to CSV
df_profile.to_csv(filename, index=False)

# Notify and trigger download
print(f"✅ File saved as {filename}")
files.download(filename)


# Imputation and Missing

| Column Type | What happens in your code                       | Notes / Limitations                                                       |
| ----------- | ----------------------------------------------- | ------------------------------------------------------------------------- |
| Categorical | `NULL` → `"__MISSING__"`                        | Good for placeholders, preserves missing info                             |
| Numeric     | `NULL` → `"__MISSING__"` (converted to string!) | ⚠️ Numeric info is lost; you probably want median/mean imputation instead |
| All columns | `_is_missing` flag created                      | Useful for autoencoder to learn missing patterns                          |


In [ ]:
select_clauses = ["*"] # Start with all original columns

for col in columns:
    # 1. Impute NULLs with a placeholder string '__MISSING__'
    imputation_clause = f"COALESCE(CAST({col} AS STRING), '__MISSING__') AS {col}_imputed"

    # 2. Create the missing flag column
    missing_flag_clause = f"CASE WHEN {col} IS NULL THEN 1 ELSE 0 END AS {col}_is_missing"

    select_clauses.append(imputation_clause)
    select_clauses.append(missing_flag_clause)

# Combine all clauses into the final SELECT part
select_statement = ",\n".join(select_clauses)

print("SELECT statement generated successfully.")

In [ ]:
final_sql_query = f"""
CREATE OR REPLACE TABLE `stgw-shared-ai-dev.parquet_data_set.IMP_profile_FE_step_2` AS
SELECT
    {select_statement}
FROM
    `{table_id}`
"""

print("Starting BigQuery Feature Engineering Job...")
# Execute the query (this will be a very large job run in BigQuery)
job = client.query(final_sql_query)
# Wait for the job to complete (you are waiting for the job to finish on the server)
job.result()

print(f"Feature engineering table created successfully: {job.destination.table_id}")

# **Sample and Extract Model Features**

| Step | Purpose                                              | Outcome                                                            |
| ---- | ---------------------------------------------------- | ------------------------------------------------------------------ |
| 1    | Decide which columns to use and drop irrelevant ones | Two lists: `_imputed` features + `_is_missing` flags               |
| 2    | Build a SQL query selecting only relevant features   | `sample_query` that samples ~5% of rows                            |
| 3    | Execute the query and save results                   | BigQuery table `IMP_profile_FE_new_sample` ready for preprocessing |


**Step 1: Load Profiling Results and Determine Columns**

Goal: Decide which columns to keep for modeling (drop IDs/constants, keep _imputed and _is_missing columns).

Explanation:

Load profiling results:

The CSV column_profiling_results_*.csv contains analysis of all columns, such as missing value percentages, uniqueness, and recommendations.

Drop unwanted columns:

Columns like IDs or constants are not informative for training an autoencoder.

drop_cols stores these column names so they won’t be used as features.

Create feature lists:

feature_cols: keeps the imputed versions of the columns (_imputed). These are your main features with nulls handled.

missing_flag_cols: keeps the missing value indicator flags (_is_missing) which help the autoencoder detect patterns of missingness.

Outcome: You now have two lists of columns that are safe and meaningful for training.

* Loaded profiling results – the CSV contains each

* column and whether it should be kept or dropped (like IDs or constants).

* Identified columns to drop – any column marked as an ID or constant in the recommendation is removed.

* Prepared final feature lists – for the remaining columns:

* _imputed columns (values after imputation) are kept as feature_cols.

* _is_missing columns (missing value flags) are kept as missing_flag_cols.





In [ ]:
import pandas as pd

# Load profiling results (generated from your earlier profiling step)
profile = pd.read_csv("/content/column_profiling_results_20251025_072916.csv")

# Identify columns to drop (IDs, constants)
drop_cols = profile.loc[
    profile["recommendation"].str.contains("drop|id", case=False, na=False),
    "column_name"
].tolist()

# Keep imputed columns
feature_cols = [f"{col}_imputed" for col in profile["column_name"] if col not in drop_cols]

# Keep missing flags
missing_flag_cols = [f"{col}_is_missing" for col in profile["column_name"] if col not in drop_cols]


**Step 2: Build a Random Sample Query**

Goal: Select only the relevant columns from IMP_profile_FE_step_2 and sample a fraction (to keep Colab memory safe).

Explanation:

Combine columns to select:

select_columns includes all _imputed features + _is_missing flags.

Generate SQL SELECT statement:

select_statement is a comma-separated string of all columns to include in the query.

Random sampling:

WHERE RAND() < 0.05 selects approximately 5% of rows randomly.

This keeps the dataset manageable in memory while preserving variability for training.

Outcome: You have a SQL query (sample_query) that selects only the relevant columns and samples a small subset of rows.

In [ ]:
select_columns = feature_cols + missing_flag_cols
select_statement = ", ".join(select_columns)

sample_query = f"""
SELECT {select_statement}
FROM `stgw-shared-ai-dev.parquet_data_set.IMP_profile_FE_step_2`
WHERE RAND() < 0.05  -- ~5% random sample
"""


**Step 3: Write Sample to a BigQuery Table**

Goal: Avoid “response too large” errors by storing the sample in a new BigQuery table first.

Explanation:

Initialize BigQuery client:

client = bigquery.Client(...) lets you run queries and manage tables in BigQuery.

Define destination table:

sample_table_id is the new table that will hold the sampled data.

WRITE_TRUNCATE ensures any existing table with the same name is replaced.

Run the query and save results:

client.query(..., job_config=job_config) executes the SQL sample_query in BigQuery.

job.result() waits for the query to finish.

Outcome:

BigQuery now has a sampled, clean, feature-ready table called IMP_profile_FE_new_sample.

This table contains only numeric-ready or imputed columns and missing value flags, ready for further preprocessing or autoencoder training.

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project="stgw-shared-ai-dev")
sample_table_id = "stgw-shared-ai-dev.parquet_data_set.IMP_profile_FE_new_sample"

job_config = bigquery.QueryJobConfig(destination=sample_table_id)
job_config.write_disposition = "WRITE_TRUNCATE"  # overwrite if exists

job = client.query(sample_query, job_config=job_config)
job.result()  # wait for the query to finish

print(f"Sample written to table: {sample_table_id}")


# **BREAK**

# **Visualisation**

Import a Sample of the Feature-Engineered Table
First, we must sample the massive table in BigQuery and load that sample into your Colab memory. We will use the approach defined earlier.

In [ ]:
import pandas as pd
from google.cloud import bigquery
import numpy as np

# Ensure your client is initialized for Colab Enterprise (as discussed)
client = bigquery.Client()
table_id = "stgw-shared-ai-dev.parquet_data_set.IMP_profile_FE_step_2" # Use the new table

# --- Determine Sampling Parameters ---
TARGET_SAMPLE_SIZE = 100000
total_rows_query = f"SELECT count(*) AS total_rows FROM `{table_id}`"
total_rows = client.query(total_rows_query).to_dataframe().iloc[0, 0]
sample_probability = TARGET_SAMPLE_SIZE / total_rows
sample_probability = min(max(sample_probability, 0.0001), 1.0)

print(f"Total Rows: {total_rows}. Sampling Probability: {sample_probability:.4f}")

# --- Execute the Sampled Query ---
query_sampled = f"""
SELECT *
FROM `{table_id}`
WHERE RAND() < {sample_probability}
LIMIT {TARGET_SAMPLE_SIZE}
"""

df_sample = client.query(query_sampled).to_dataframe()

print(f"\nSuccessfully imported a sample of {len(df_sample)} rows.")
print(f"Sampled DataFrame shape: {df_sample.shape}")

 Visualization of Missingness (Binary Features)
This plot validates your *_is_missing columns and shows which original features had the highest rates of missing data.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Identify all the binary missing flag columns
missing_cols = [col for col in df_sample.columns if col.endswith('_is_missing')]

# Calculate the mean of these columns (which gives the proportion of missing values)
missing_rate = df_sample[missing_cols].mean().sort_values(ascending=False)

# Plot the top 20 most missing features
plt.figure(figsize=(12, 6))
missing_rate.head(20).plot(kind='bar', color='skyblue')
plt.title('Top 20 Features by Missing Value Rate')
plt.ylabel('Proportion Missing (0 to 1)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show() #

print("Visualized the proportion of missing data in the original features.")

Inspecting the Sampled Columns
Run this code in Colab to see the names of all the new feature-engineered columns in your df_sample DataFrame:

In [ ]:
# Identify the two types of feature-engineered columns
imputed_cols = sorted([col for col in df_sample.columns if col.endswith('_imputed')])
missing_flag_cols = sorted([col for col in df_sample.columns if col.endswith('_is_missing')])

print("--- 1. Imputed Columns (String/Categorical) ---")
print(f"Total: {len(imputed_cols)}")
# Print the first 10 for a quick look
print(imputed_cols[:10])

print("\n--- 2. Missing Flag Columns (Binary 0/1) ---")
print(f"Total: {len(missing_flag_cols)}")
# Print the first 10 for a quick look
print(missing_flag_cols[:10])

Choosing and Visualizing Columns
Once you've reviewed the lists above, you can replace the placeholder names in sample_imputed_cols with actual columns that are most important or that showed high missingness in the previous step (e.g., from the "Top 20 Features by Missing Value Rate" chart).

In [ ]:
import matplotlib.pyplot as plt

# Replace these with actual column names you chose from the lists above
sample_imputed_cols = [
    'activities_imputed',
    'adults_income_imputed', # Replace with an actual imputed column from your list
    'age_imputed'    # Replace with another actual imputed column
]

for col in sample_imputed_cols:
    if col in df_sample.columns:
        plt.figure(figsize=(8, 4))
        # Plot the top 10 value counts, including the '__MISSING__' placeholder
        df_sample[col].value_counts().head(10).plot(kind='barh', color='lightcoral')
        plt.title(f'Distribution for {col}')
        plt.show() #
    else:
        print(f"Warning: Column '{col}' not found in the sample DataFrame.")

4. Correlation of Missingness Patterns
This plot validates the relationships between your new binary features. It shows if some features tend to be missing at the same time.

In [ ]:
# Create a DataFrame only with the missing flag columns
df_missing_flags = df_sample[missing_cols]

# Calculate the correlation matrix
# Due to the high number of columns (227), we sample a subset for the heatmap
sample_cols = df_missing_flags.sample(n=30, axis=1, random_state=42)
corr_matrix_missing = sample_cols.corr()

plt.figure(figsize=(10, 10))
sns.heatmap(corr_matrix_missing, cmap='viridis', annot=False, linewidths=.5, cbar_kws={'label': 'Correlation (0 to 1)'})
plt.title('Correlation Heatmap of Sampled Missingness Flags (0=No Correlation, 1=Missing Together)')
plt.show() #

print("Visualized patterns in features missing simultaneously.")

**2. Feature Matrix Creation and Dimensionality Reduction (BigQuery ML)Goal:**
Convert $\approx 681$ high-cardinality categorical features into a small, dense, numeric feature matrix. This is the most crucial step for clustering categorical data.A. Dimensionality Reduction via Feature EmbeddingsThe most effective method for "latent behavioral patterns" is to use Feature Embeddings via a BQML model like Autoencoder or Matrix Factorization. This converts your wide categorical table into a narrow numeric table.

1. Revised BQML Query: Implementing Sampling (Recommended)
You can sample the data directly within the BigQuery SQL query using the WHERE RAND() < X clause. A small sample (e.g., 1 million rows, or about 0.4% of your data) is often sufficient for initial model training and evaluation.

In [ ]:
import time
from google.cloud import bigquery
from google.cloud.bigquery import QueryJobConfig # Imported but not strictly needed for this simple query

# -------------------------------------------------------------
# 0. Setup and Query Definition
# -------------------------------------------------------------
PROJECT_ID = 'stgw-shared-ai-dev'
LOCATION = 'us-west1' # Using us-west1 to match the location error encountered previously

# Initialize the BigQuery client
client = bigquery.Client(project=PROJECT_ID, location=LOCATION)

# Use a sampling rate (e.g., 0.005 = 0.5% of 261M rows ≈ 1.3 million rows)
SAMPLING_RATE = 0.005

bqml_query = f"""
CREATE OR REPLACE MODEL
  `stgw-shared-ai-dev.parquet_data_set.behavioral_autoencoder_SAMPLE`
OPTIONS(
  model_type='AUTOENCODER',
  HIDDEN_UNITS=[100, 50, 100],
  OPTIMIZER='ADAM',
  MAX_ITERATIONS=3
) AS
SELECT
    * EXCEPT(
    extern_tuid,       -- ID column
    created_date,      -- Unsupported TIMESTAMP type
    dateOfBirth        -- Unsupported DATE/TIMESTAMP type
)
FROM
    `stgw-shared-ai-dev.parquet_data_set.IMP_profile_FE_step_2`
WHERE
    RAND() < {SAMPLING_RATE}  -- Implementing the sampling
"""

print(f"Starting sampled BQML Autoencoder Training Job in {LOCATION}...")

# -------------------------------------------------------------
# 1. Start the BigQuery job
# -------------------------------------------------------------
job = client.query(bqml_query)
print(f"Job started successfully. ID: {job.job_id}")

# -------------------------------------------------------------
# 2. Asynchronous Polling Loop (Mitigation for Runtime Disconnects)
# -------------------------------------------------------------
MAX_POLL_TIME_MINUTES = 360 # 6 hours max duration
poll_interval_seconds = 600 # Check status every 10 minutes

start_time = time.time()
while job.state != 'DONE' and (time.time() - start_time) < (MAX_POLL_TIME_MINUTES * 60):
    time.sleep(poll_interval_seconds)

    # Reload the job status from BigQuery
    job.reload()

    elapsed = int((time.time() - start_time) / 60)

    # Print status to keep the notebook active
    print(f"Job status: {job.state} | Elapsed: {elapsed} min. (Checking again in {poll_interval_seconds/60} min)")

# -------------------------------------------------------------
# 3. Final Check and Output
# -------------------------------------------------------------
if job.error_result:
    # If the job failed on the BigQuery side
    raise Exception(f"BigQuery Job Failed: {job.error_result['reason']}. Check BQ Console for details.")
elif job.state == 'DONE':
    # If the job completed successfully
    print(f"\nAutoencoder Model created successfully: {job.destination.table_id} in {LOCATION}")
else:
    # If the polling loop timed out before BigQuery was finished
    print(f"\nPOLLING TIMEOUT: Job is still running on BigQuery but the notebook stopped waiting. Job ID: {job.job_id}")
    print("You can check the final status directly in the BigQuery UI using the Job ID.")

In [ ]:
from google.cloud import bigquery
from google.colab import auth

# Initialize the BigQuery client (adjust location if necessary)
PROJECT_ID = 'stgw-shared-ai-dev'
TABLE_ID_BASE = "stgw-shared-ai-dev.parquet_data_set.IMP_profile_joined_behavioral_base"

# Ensure client is initialized (authentication is assumed in Colab Enterprise)
client = bigquery.Client(project=PROJECT_ID)

# 1. Query BigQuery's information schema to get column names
query_columns = f"""
SELECT column_name
FROM `{TABLE_ID_BASE.split('.')[0]}`.{TABLE_ID_BASE.split('.')[1]}.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = '{TABLE_ID_BASE.split('.')[-1]}'
"""

# Store the list of 227 column names in the 'columns' variable
columns = [row['column_name'] for row in client.query(query_columns).result()]

print(f"Successfully fetched {len(columns)} original column names.")

In [ ]:
# Assuming 'columns' is your list of 227 original column names
# If 'columns' is not defined, you must run the initial query to fetch them first.

# 1. Manually define the list of original columns if you don't have the 'columns' variable
# Example: columns = ['age', 'country', 'feature_X', ...]

clean_features = []
# Generate the names of the clean features (approx. 681 names)
for col in columns:
    clean_features.append(f"{col}_imputed")
    clean_features.append(f"{col}_is_missing")

# Add the ID column, which must be the first column
clean_features.insert(0, "extern_tuid")

# Final comma-separated string for the SQL SELECT clause
clean_select_statement = ",\n".join(clean_features)

print("--- COPY THE TEXT BELOW AND PASTE IT INTO BIGQUERY ---")
print(clean_select_statement)
print("-------------------------------------------------------")

In [ ]:
# Assuming 'columns' is your list of 227 original column names (without extern_tuid, created_date, dateOfBirth)

clean_features = []
# Only generate the imputed columns
for col in columns:
    # Cast all imputed features to STRING to avoid type mismatches
    clean_features.append(f"CAST({col}_imputed AS STRING) AS {col}_imputed")
    # Skip the _is_missing flag features for simplicity and reduction

# Add the ID column, which remains as a string/integer
clean_features.insert(0, "extern_tuid")

# Final comma-separated string for the SQL SELECT clause
clean_select_statement_REDUCED = ",\n".join(clean_features)

print("--- COPY THIS NEW REDUCED TEXT BELOW AND PASTE IT INTO BIGQUERY ---")
print(clean_select_statement_REDUCED)
print("-------------------------------------------------------")

In [ ]:
import pandas as pd
from google.cloud import bigquery
# Assume 'client' is your initialized BigQuery client object
# client = bigquery.Client()

# Define the SQL query string
preview_query = f"""
SELECT
    extern_tuid,
    -- Examples of imputed string features:
    CAST(credit_tier_imputed AS STRING) AS credit_tier_imputed,
    CAST(family_income_decile_imputed AS STRING) AS family_income_decile_imputed,
    CAST(timezone_imputed AS STRING) AS timezone_imputed,
    -- Examples of imputed high-cardinality features:
    CAST(emailAddress_imputed AS STRING) AS emailAddress_imputed,
    CAST(i_domains_imputed AS STRING) AS i_domains_imputed,
    -- Examples of imputed numeric features (check if they became strings):
    CAST(age_imputed AS STRING) AS age_imputed,
    CAST(number_of_children_imputed AS STRING) AS number_of_children_imputed,
    -- Additional columns:
    CAST(credit_card_details_imputed AS STRING) AS credit_card_details_imputed,
    CAST(home_ownership_imputed AS STRING) AS home_ownership_imputed
FROM
    `stgw-shared-ai-dev.parquet_data_set.IMP_profile_FE_SAMPLE`
LIMIT 10;
"""

print("Running preview query in BigQuery...")

# Execute the query and load the results into a pandas DataFrame
df_preview = client.query(preview_query).to_dataframe()

print("Preview Results (First 10 Rows):")
# Display the DataFrame
display(df_preview)

# **CRAMER V'S ANALYSIS**

In [ ]:
from google.cloud import bigquery

# Set the project ID explicitly if the client doesn't pick it up automatically
PROJECT_ID = 'stgw-shared-ai-dev'

# The BigQuery client should automatically use the Service Account
# associated with your Colab Enterprise notebook/runtime.
client = bigquery.Client(project=PROJECT_ID)

print(f"BigQuery Client initialized for project: {PROJECT_ID}")
# You can now proceed to run your BigQuery queries (CREATE TABLE, SELECT, etc.)

In [ ]:
import pandas as pd
from google.cloud import bigquery
from scipy.stats import chi2_contingency
import numpy as np

# Define the project and dataset for convenience
PROJECT_ID = 'stgw-shared-ai-dev'
DATASET_ID = 'parquet_data_set'
SAMPLE_TABLE = 'IMP_profile_FE_SAMPLE'

# --- Construct the list of imputed columns for the query ---
# You need the list of your original 227 column names (without extern_tuid, etc.)
# If you have the 'columns' list from previous steps, use it. Otherwise, manually list them.
# Example:
# imputed_cols = [f"{col}_imputed" for col in columns]
# imputed_cols.insert(0, 'extern_tuid') # Add the ID column

# For this example, let's manually pick a few features to demonstrate:
imputed_cols = [
    'mortgage_loan_imputed',
    'credit_cards_used_most_imputed',
    'auto_loan_imputed',
    'Unlock_gender_imputed',
    'credit_tier_imputed',
    'family_income_decile_imputed',
    'average_household_income_imputed',
    'financial_beliefs_imputed',
    'loan_imputed',
    'credit_card_spend_imputed',
    'credit_card_balance_imputed',
    'equity_balance_imputed',
    'family_level_of_investable_assets_imputed',
    'net_worth_imputed',
    'credit_cards_used_30days_imputed',
    'card_type_imputed'
]
column_list_str = ",\n".join(imputed_cols)

# Load the data
print("Loading sampled data from BigQuery for correlation check...")
sample_load_query = f"""
SELECT
    {column_list_str}
FROM
    `{PROJECT_ID}.{DATASET_ID}.{SAMPLE_TABLE}`
"""

# The following line assumes 'client' is your BigQuery client
df_sample = client.query(sample_load_query).to_dataframe()
print(f"Data loaded with {len(df_sample)} rows and {len(imputed_cols) - 1} features.")

**Cramer V's function**

In [ ]:
def cramers_v(x, y):
    """Calculate Cramer's V for two categorical series."""
    confusion_matrix = pd.crosstab(x, y)

    # Chi-squared test
    chi2 = chi2_contingency(confusion_matrix)[0]

    # Number of observations
    n = confusion_matrix.sum().sum()

    # Degrees of freedom for rows and columns
    r, k = confusion_matrix.shape

    # Calculate Cramer's V
    phi2 = chi2 / n
    # Bias correction (optional but recommended)
    # The term min(k-1, r-1) is the effective degrees of freedom
    phi2_corrected = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    r_corrected = r - ((r-1)**2)/(n-1)
    k_corrected = k - ((k-1)**2)/(n-1)

    # Final Cramer's V (bias-corrected)
    v = np.sqrt(phi2_corrected / min(r_corrected - 1, k_corrected - 1))

    # Edge case: If the matrix has only one row/column (e.g., after filtering NULLs)
    if np.isclose(min(r_corrected - 1, k_corrected - 1), 0):
        return 1.0 # Or handle as an error/skip

    return v

# List of only feature columns (excluding the ID)
feature_cols = [col for col in imputed_cols if col != 'extern_tuid']

In [ ]:
print("Calculating Cramer's V Matrix...")

# Create an empty DataFrame for the Cramer's V matrix
cramer_matrix = pd.DataFrame(index=feature_cols, columns=feature_cols, dtype=float)

for i in range(len(feature_cols)):
    for j in range(i, len(feature_cols)):
        col1 = feature_cols[i]
        col2 = feature_cols[j]

        # Calculate V only once (matrix is symmetric)
        if i == j:
            cramer_matrix.loc[col1, col2] = 1.0
        else:
            # We must use the .astype(str) here to ensure all inputs are treated as categories
            v = cramers_v(df_sample[col1].astype(str), df_sample[col2].astype(str))
            cramer_matrix.loc[col1, col2] = v
            cramer_matrix.loc[col2, col1] = v

print("Cramer's V Matrix (Association Strength):")
# display(cramer_matrix) # Uncomment this to see the full matrix

In [ ]:
# Identify highly correlated pairs (V > 0.95)
HIGH_CORRELATION_THRESHOLD = 0.95
redundant_pairs = []

# Loop through the upper triangle (excluding the diagonal)
for i in range(len(feature_cols)):
    for j in range(i + 1, len(feature_cols)):
        col1 = feature_cols[i]
        col2 = feature_cols[j]
        v_score = cramer_matrix.loc[col1, col2]

        if v_score > HIGH_CORRELATION_THRESHOLD:
            redundant_pairs.append((v_score, col1, col2))

if redundant_pairs:
    print("\n🚨 Highly Redundant Feature Pairs (V > 0.95):")
    for score, col_a, col_b in sorted(redundant_pairs, reverse=True):
        print(f"  V={score:.4f}: {col_a} vs {col_b}")

    print("\nRecommendation: Remove one column from each pair listed above.")
else:
    print("\n✅ No highly redundant feature pairs (V > 0.95) found in the sample.")

**CREATE CRAMER V's FUNCTION ON ENTIRE DATASET**

In [ ]:
import pandas as pd
from google.cloud import bigquery
from scipy.stats import chi2_contingency
import numpy as np

# --- 1. CONFIGURATION ---
PROJECT_ID = 'stgw-shared-ai-dev'
DATASET_ID = 'parquet_data_set'
SAMPLE_TABLE = 'IMP_profile_FE_SAMPLE'
# Assume 'client' is your initialized BigQuery client object

# --- 2. DYNAMICALLY RETRIEVE FEATURE LIST FROM BIGQUERY SCHEMA ---

print("Step 1/4: Retrieving the full list of imputed columns...")

# Query the INFORMATION_SCHEMA to get all columns ending in '_imputed' + 'extern_tuid'
schema_query = f"""
SELECT column_name
FROM `{PROJECT_ID}.{DATASET_ID}.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = '{SAMPLE_TABLE}'
  AND column_name LIKE '%_imputed'
ORDER BY column_name
"""
# The 'extern_tuid' is added separately as it's the ID, not a feature
feature_list_df = client.query(schema_query).to_dataframe()

# Create the final list of features to analyze
imputed_feature_cols = feature_list_df['column_name'].tolist()
imputed_feature_cols.insert(0, 'extern_tuid')

column_list_str = ",\n".join(imputed_feature_cols)
print(f"✅ Retrieved {len(imputed_feature_cols) - 1} imputed features.")


# --- 3. LOAD THE FULL SAMPLED DATASET ---

print("Step 2/4: Loading the full sampled data into DataFrame...")
sample_load_query = f"""
SELECT
    {column_list_str}
FROM
    `{PROJECT_ID}.{DATASET_ID}.{SAMPLE_TABLE}`
"""

# Load the data, which should be about 1.3 million rows
df_sample = client.query(sample_load_query).to_dataframe()
print(f"✅ Data loaded with {len(df_sample)} rows and {len(imputed_feature_cols)} columns.")


# --- 4. CRAMER'S V CALCULATION FUNCTION ---

print("Step 3/4: Defining Cramer's V function...")
def cramers_v(x, y):
    """Calculate Cramer's V for two categorical series."""
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape

    # Calculate Cramer's V (with bias correction)
    phi2 = chi2 / n
    phi2_corrected = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    r_corrected = r - ((r-1)**2)/(n-1)
    k_corrected = k - ((k-1)**2)/(n-1)

    # Avoid division by zero
    min_denom = min(r_corrected - 1, k_corrected - 1)
    if np.isclose(min_denom, 0):
        return 1.0 # Perfect correlation for this edge case

    return np.sqrt(phi2_corrected / min_denom)

feature_cols_only = [col for col in imputed_feature_cols if col != 'extern_tuid']


# --- 5. GENERATE & FILTER CORRELATION MATRIX ---

print("Step 4/4: Calculating and filtering Cramer's V Matrix...")

# Initialize matrix
cramer_matrix = pd.DataFrame(index=feature_cols_only, columns=feature_cols_only, dtype=float)

# Calculate V
for i in range(len(feature_cols_only)):
    for j in range(i, len(feature_cols_only)):
        col1 = feature_cols_only[i]
        col2 = feature_cols_only[j]

        if i == j:
            cramer_matrix.loc[col1, col2] = 1.0
        else:
            # Cast all features to string for safety before calculating V
            v = cramers_v(df_sample[col1].astype(str), df_sample[col2].astype(str))
            cramer_matrix.loc[col1, col2] = v
            cramer_matrix.loc[col2, col1] = v


# Identify highly redundant pairs (V > 0.95)
HIGH_CORRELATION_THRESHOLD = 0.95
redundant_pairs = []

for i in range(len(feature_cols_only)):
    for j in range(i + 1, len(feature_cols_only)):
        score = cramer_matrix.iloc[i, j]
        if score > HIGH_CORRELATION_THRESHOLD:
            redundant_pairs.append((score, feature_cols_only[i], feature_cols_only[j]))

print("\n" + "="*50)
if redundant_pairs:
    print("🚨 HIGHLY REDUNDANT FEATURE PAIRS (Cramer's V > 0.95):")
    for score, col_a, col_b in sorted(redundant_pairs, reverse=True):
        print(f"  V={score:.4f}: {col_a} vs {col_b}")

    print("\nACTION: For each pair, choose one to remove from your final training list.")
else:
    print("✅ NO HIGHLY REDUNDANT FEATURE PAIRS (V > 0.95) FOUND.")

print("="*50)

# **CRAMER V FOR TARGETED COLUMNS**

In [ ]:
import pandas as pd
from google.cloud import bigquery
from scipy.stats import chi2_contingency
import numpy as np

# --- CONFIGURATION ---
PROJECT_ID = 'stgw-shared-ai-dev'
DATASET_ID = 'parquet_data_set'
SAMPLE_TABLE = 'IMP_profile_FE_SAMPLE'

# Initialize the BigQuery client
client = bigquery.Client(project=PROJECT_ID)

# --- 1. DEFINE CRAMER'S V FUNCTION ---
def cramers_v(x, y):
    """Calculate Cramer's V for two categorical series."""
    # NOTE: pd.crosstab is the line that generates the memory warning/crash
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape

    # Calculate Cramer's V (with bias correction)
    phi2 = chi2 / n
    phi2_corrected = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    r_corrected = r - ((r-1)**2)/(n-1)
    k_corrected = k - ((k-1)**2)/(n-1)

    min_denom = min(r_corrected - 1, k_corrected - 1)
    if np.isclose(min_denom, 0):
        return 1.0

    return np.sqrt(phi2_corrected / min_denom)


# --- 2. DEFINE FEATURE LISTS (Cleaned for Training) ---

TARGETED_FEATURE_LIST_CLEAN = [
    'mortgage_loan_imputed', 'credit_cards_used_most_imputed', 'auto_imputed',
    'auto_loan_imputed', 'Unlock_gender_imputed', 'credit_tier_imputed',

    # Grocery Items (22 Remaining)
    'grocery_tortilla_chips_imputed', 'grocery_cold_cereal_imputed', 'grocery_hot_cereal_imputed',
    'grocery_ice_cream_imputed', 'grocery_tea_imputed', 'grocery_food_snack_dessert_imputed',
    'grocery_mustard_imputed', 'grocery_beverages_nonalcoholic_drinks_imputed',
    'grocery_rice_imputed', 'grocery_potato_chips_imputed', 'grocery_chocolate_candy_imputed',
    'grocery_peanut_butter_imputed', 'grocery_pasta_imputed', 'grocery_nutritional_snacks_imputed',
    'grocery_breads_imputed', 'grocery_iced_tea_imputed',
    'grocery_stores_imputed', 'grocery_bbq_seasoning_sauces_imputed', 'grocery_soft_drinks_imputed',
    'grocery_canned_soups_imputed', 'grocery_hot_dogs_imputed', 'grocery_crackers_imputed',

    # Financial/Credit Features (17 Remaining)
    'family_income_decile_imputed', 'average_household_income_imputed',
    'financial_beliefs_imputed', 'loan_imputed', 'life_insurance_imputed',
    'credit_behavior_imputed', 'credit_card_spend_imputed',
    'credit_card_balance_imputed', 'adults_income_imputed', 'equity_balance_imputed',
    'family_level_of_investable_assets_imputed', 'net_worth_imputed',
    'home_equity_imputed', 'investments_type_imputed', 'charitable_contributions_imputed',
    'credit_cards_used_30days_imputed', 'card_type_imputed',

    # Technology/Sites (19 Remaining)
    'tech_learning_imputed', 'streaming_system_device_use_imputed',
    'retail_sites_imputed', 'batteries_imputed', 'tv_brand_owned_imputed',
    'hh_cable_subscriptions_imputed', 'cell_smartphones_tablets_imputed', 'downloaded_streamed_from_internet_imputed',
    'camera_brand_owned_imputed', 'news_sites_imputed', 'technology_imputed',
    'residential_isp_imputed', 'social_sites_imputed',
    'tv_when_acquired_imputed', 'tablets_imputed',
    'sports_sites_imputed', 'travel_sites_imputed', 'mobile_service_provider_imputed', 'radio_imputed',
    'i_device_types_imputed', 'i_iab_categories_imputed', 'i_device_oss_imputed', # Re-added i_device_oss_imputed to keep one OS feature
    'cable_sat_provider_imputed'
]

# The list used for loading data (includes the ID)
TARGETED_FEATURE_LIST_LOAD = ['extern_tuid'] + TARGETED_FEATURE_LIST_CLEAN


# --- 3. LOAD TARGETED DATA ---
column_list_str = ",\n".join(TARGETED_FEATURE_LIST_LOAD)

sample_load_query = f"""
SELECT {column_list_str}
FROM `{PROJECT_ID}.{DATASET_ID}.{SAMPLE_TABLE}`
"""

print("Loading extended targeted data for Cramer's V check...")
# Load the data
df_target = client.query(sample_load_query).to_dataframe()
print(f"Loaded {len(df_target)} rows with {len(TARGETED_FEATURE_LIST_CLEAN)} features.")


# --- 4. GENERATE & FILTER CORRELATION MATRIX ---
feature_cols_only = TARGETED_FEATURE_LIST_CLEAN
HIGH_CORRELATION_THRESHOLD = 0.90
redundant_pairs = []

print(f"Calculating targeted Cramer's V scores (approx. {len(feature_cols_only)*(len(feature_cols_only)-1)//2} pairs)...")

# Monitor your memory and runtime carefully during this loop!
for i in range(len(feature_cols_only)):
    for j in range(i + 1, len(feature_cols_only)):
        col1 = feature_cols_only[i]
        col2 = feature_cols_only[j]

        # Calculate V score (This is the high-memory operation)
        v_score = cramers_v(df_target[col1].astype(str), df_target[col2].astype(str))

        if v_score > HIGH_CORRELATION_THRESHOLD:
            redundant_pairs.append((v_score, col1, col2))

# --- 5. OUTPUT RESULTS ---
print("\n" + "="*70)
if redundant_pairs:
    print(f"🚨 HIGHLY REDUNDANT FEATURE PAIRS (Cramer's V > {HIGH_CORRELATION_THRESHOLD:.2f}):")
    for score, col_a, col_b in sorted(redundant_pairs, reverse=True):
        print(f"  V={score:.4f}: {col_a} vs {col_b}")

    print("\nACTION: For each pair, choose one to remove from your final training list.")
else:
    print("✅ No highly redundant pairs (V > 0.90) found in the extended targeted list.")

print("="*70)

2. Final Training Query (Once Parameters are Tuned)
Once you are satisfied with the performance and configuration of your sampled model, you can run the full job on the entire 261M rows by simply removing the sampling clause:

In [ ]:
FINAL_BEHAVIORAL_FEATURE_LIST_50 = [
    'credit_tier_imputed', 'credit_card_details_imputed', 'family_income_decile_imputed',
    'average_household_income_imputed', 'credit_card_spend_imputed', 'equity_balance_imputed',
    'net_worth_imputed', 'credit_cards_used_most_imputed', 'discretionary_spend_imputed',
    'tv_brand_owned_imputed', 'camera_brand_owned_imputed', 'i_location_brands_imputed',
    'gas_brand_imputed', 'vehicle_make_imputed', 'drink_liquor_brand_imputed',
    'mobile_phone_brand_by_carrier_imputed', 'purchase_beer_brand_imputed',
    'purchase_wine_brand_imputed', 'drink_beer_brand_imputed', 'credit_cards_company_imputed',
    'shoppers_apparel_purchase_last_12_mnths_imputed', 'cruises_imputed',
    'fast_food_and_drive_in_restaurant_imputed', 'leisure_activities_hobbies_imputed',
    'auto_parts_auto_repair_imputed', 'clothing_imputed', 'magazines_imputed',
    'books_imputed', 'shopping_imputed', 'travel_imputed', 'hotels_for_any_travel_imputed',
    'commerce_retail_shopping_imputed', 'family_restaurant_and_steak_house_imputed',
    'department_discount_warehouse_imputed', 'mail_purchases_imputed', 'shoes_imputed',
    'television_sites_imputed', 'streaming_system_device_use_imputed', 'retail_sites_imputed',
    'entertainment_sites_imputed', 'news_sites_imputed', 'travel_sites_imputed',
    'social_sites_imputed', 'grocery_tortilla_chips_imputed', 'grocery_cold_cereal_imputed',
    'grocery_ice_cream_imputed', 'grocery_tea_imputed', 'grocery_food_snack_dessert_imputed',
    'grocery_chocolate_candy_imputed', 'grocery_stores_imputed'
]

# --- CONSTRUCT THE FINAL SELECT CLAUSE (With CAST) ---
feature_select_clauses = ["extern_tuid"]
for col in FINAL_BEHAVIORAL_FEATURE_LIST_50:
    feature_select_clauses.append(f"CAST({col} AS STRING) AS {col}")

clean_select_statement_50 = ",\n".join(feature_select_clauses)

print("Starting Autoencoder training with 50 highly-focused behavioral features...")

bqml_training_query_50 = f"""
CREATE OR REPLACE MODEL
  `stgw-shared-ai-dev.parquet_data_set.behavioral_autoencoder_50F`
OPTIONS(
  model_type='AUTOENCODER',
  HIDDEN_UNITS=[100, 50, 100],
  OPTIMIZER='ADAM',
  MAX_ITERATIONS=1,
  AUTO_CATEGORY_HASH=TRUE
) AS
SELECT
    {clean_select_statement_50}
FROM
    `stgw-shared-ai-dev.parquet_data_set.IMP_profile_FE_SAMPLE`;
"""

B. Create the Numeric Feature Matrix
Use the trained model to generate the new, reduced features for every row.

In [ ]:
from google.cloud import bigquery

# Assuming client is initialized and Autoencoder training was successful.
PROJECT_ID = 'stgw-shared-ai-dev'
LOCATION = 'us-west1'
client = bigquery.Client(project=PROJECT_ID, location=LOCATION)

# **IMPORTANT: We use extern_tuid as the join key**
bqml_generate_query = """
CREATE OR REPLACE TABLE
  `stgw-shared-ai-dev.parquet_data_set.final_feature_matrix` AS
SELECT
  t1.extern_tuid, -- Use the correct ID column from the source table (t1)
  t2.feature_vector AS embeddings
FROM
  `stgw-shared-ai-dev.parquet_data_set.IMP_profile_FE_step_2` AS t1
JOIN
  ML.GENERATE_EMBEDDING(
    MODEL `stgw-shared-ai-dev.parquet_data_set.behavioral_autoencoder`,
    (SELECT * EXCEPT(extern_tuid, created_date) FROM `stgw-shared-ai-dev.parquet_data_set.IMP_profile_FE_step_2`)
  ) AS t2
ON t1.extern_tuid = t2.extern_tuid;
"""

print("Starting BigQuery job to generate feature embeddings...")
job = client.query(bqml_generate_query)
job.result()
print(f"Feature matrix created successfully at: {job.destination.table_id}")

3. Correlation Plot and Clustering (Colab/Scikit-learn)
A. Import the Reduced Feature Matrix
We import the small final_feature_matrix table and convert the single embeddings column (which contains an array of 50 latent features) into a standard DataFrame for scikit-learn.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from google.cloud import bigquery

# Re-initialize client for clarity
client = bigquery.Client()
final_table_id = "stgw-shared-ai-dev.parquet_data_set.final_feature_matrix"

# Import the final, reduced feature matrix
df_features = client.query(f"SELECT * FROM `{final_table_id}`").to_dataframe()

# Convert the 'embeddings' list/array column into a DataFrame of individual features
X = pd.DataFrame(df_features['embeddings'].to_list(), index=df_features.index)
extern_tuids = df_features['extern_tuid'] # Store IDs separately

print(f"Imported feature matrix shape: {X.shape}")

B. Correlation Plot
Visualize the relationships between the 50 learned latent features.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

corr_matrix = X.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm')
plt.title('Correlation Plot of 50 Latent Feature Embeddings')
plt.show()

C. Meaningful Clustering (K-Means)We perform K-Means clustering after determining the optimal $K$ (clusters) via the Elbow or Silhouette methods (not shown here, but necessary in practice).

In [ ]:
K = 5 # Example: Assume 5 clusters is optimal
kmeans = KMeans(n_clusters=K, random_state=42, n_init='auto')
cluster_labels = kmeans.fit_predict(X)

# Attach clusters back to the IDs
df_results = pd.DataFrame({
    'extern_tuid': extern_tuids,
    'cluster': cluster_labels
})

print(f"Clustering complete. Results shape: {df_results.shape}")
print(df_results.head())

4. SHAP and Interpretation (BigQuery & Colab)A. SHAP (Feature Importance/Contribution)We treat the cluster label as a target variable and train a classifier (Random Forest) on the reduced features ($X$) to see which features drive the cluster assignment.

In [ ]:
import shap
from sklearn.ensemble import RandomForestClassifier

# 1. Train a classifier to predict the cluster label
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X, cluster_labels)

# 2. Calculate SHAP values for the classifier
explainer = shap.TreeExplainer(classifier)
shap_values = explainer.shap_values(X)

# 3. Plot summary (shows which of the 50 latent features are most important)
print("\nSHAP Summary Plot:")
# shap_values is a list of arrays (one for each class/cluster), we plot the absolute mean
shap.summary_plot(shap_values, X, plot_type="bar")

B. Meaningful Interpretation (Profiling in BigQuery)
This is the most critical step for business understanding. We push the cluster results back to BigQuery and join them to the original data to profile the clusters based on the original categories.

Upload Results to BigQuery:

In [ ]:
# Upload the cluster assignments to a new BigQuery table
table_id_results = "stgw-shared-ai-dev.parquet_data_set.final_cluster_assignments"
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")

client.load_table_from_dataframe(
    df_results, table_id_results, job_config=job_config
).result()

print(f"Cluster assignments uploaded to: {table_id_results}")

Profile Clusters (Example Query):

This query must be run in BigQuery (via Python) for each cluster and for multiple key original columns to define the behavioral patterns.

In [ ]:
# Example: Find the most frequent category for a specific original column
# (e.g., 'country') within Cluster 1.

cluster_profile_query = """
SELECT
  t2.country, -- Use an original column from your source data
  COUNT(t2.country) AS count_in_cluster,
  COUNT(t2.country) / SUM(COUNT(*)) OVER () AS proportion_in_cluster
FROM
  `stgw-shared-ai-dev.parquet_data_set.final_cluster_assignments` AS t1
JOIN
  `stgw-shared-ai-dev.parquet_data_set.IMP_profile_joined_behavioral_base` AS t2
ON
  t1.extern_tuid = t2.extern_tuid
WHERE t1.cluster = 1
GROUP BY 1
ORDER BY count_in_cluster DESC
LIMIT 5
"""

df_profile = client.query(cluster_profile_query).to_dataframe()
print("\nCluster 1 Profile (Top 5 countries):")
print(df_profile)

# **CORRRELATION**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.cloud import bigquery
from scipy.stats import chi2_contingency

# --- Configuration ---
PROJECT_ID = 'stgw-shared-ai-dev'
DATASET_ID = 'parquet_data_set'
SAMPLE_TABLE = 'IMP_profile_FE_SAMPLE'

# Initialize BigQuery Client
client = bigquery.Client(project=PROJECT_ID)
print(f"BigQuery Client initialized for project: {PROJECT_ID}")

# --- 1. Define the 9 Features to Analyze ---
# This list combines your 3 'kept' features with 6 logical 'dropped' candidates
# to demonstrate redundancy.
features_to_analyze = [
    # 16 Kept Features (from core_financial_cols)
    'mortgage_loan_imputed',
    'credit_tier_imputed',
    'card_type_imputed',
    # 6 Dropped/Redundant Candidates
    'mortgage_imputed',            # Redundant with mortgage_loan_imputed
    'credit_cards_imputed',        # Redundant with credit_cards_used_most/30days
    'discretionary_spend_imputed', # Redundant with credit_card_spend/balance
    'wealth_rating_imputed',       # Redundant with net_worth/family_level_of_investable_assets
    'credit_behavior_imputed',     # Redundant with credit_tier
    'personal_finance_profile_imputed' # Redundant with financial_beliefs/credit_behavior
]

column_list_str = ",\n".join(features_to_analyze)

# --- 2. Define Cramér's V Function ---
def cramers_v(x, y):
    """Calculates Cramér's V, a measure of association between two nominal variables."""
    confusion_matrix = pd.crosstab(x, y)
    if confusion_matrix.empty or confusion_matrix.shape[0] == 1 or confusion_matrix.shape[1] == 1:
        return 0.0

    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape

    phi2 = chi2 / n
    # Bias correction for categorical variables
    phi2_corrected = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    r_corrected = max(1, r - ((r-1)**2)/(n-1))
    k_corrected = max(1, k - ((k-1)**2)/(n-1))

    v = np.sqrt(phi2_corrected / min(r_corrected, k_corrected))
    return min(1.0, max(0.0, v))

# --- 3. Load Data from BigQuery ---
print(f"Loading data for {len(features_to_analyze)} features from BigQuery table: {SAMPLE_TABLE}...")

# Add a LIMIT clause for safety/speed if you want to sample, but running on full table is ideal.
sample_load_query = f"""
SELECT
    {column_list_str}
FROM
    `{PROJECT_ID}.{DATASET_ID}.{SAMPLE_TABLE}`
"""

# Fetch the data into a Pandas DataFrame
try:
    df_original = client.query(sample_load_query).to_dataframe()
    print(f"Data loaded successfully with {len(df_original)} rows.")
except Exception as e:
    print(f"Error loading data from BigQuery: {e}")
    # Create a dummy DataFrame if load fails, to show the heatmap structure
    df_original = pd.DataFrame(index=np.arange(1), columns=features_to_analyze)


# --- 4. Calculate Cramér's V Matrix ---
print("Calculating Cramér's V Matrix from original data...")
v_matrix_original = pd.DataFrame(index=features_to_analyze,
                                 columns=features_to_analyze,
                                 dtype=float)

for i in range(len(features_to_analyze)):
    for j in range(i, len(features_to_analyze)):
        col1 = features_to_analyze[i]
        col2 = features_to_analyze[j]

        # Ensure data is treated as categorical (string type)
        v = cramers_v(df_original[col1].astype(str), df_original[col2].astype(str))
        v_matrix_original.loc[col1, col2] = v
        v_matrix_original.loc[col2, col1] = v # Symmetric matrix

# --- 5. Generate and Save Heatmap ---
plt.figure(figsize=(18, 16))
sns.heatmap(
    v_matrix_original,
    annot=True,      # Show V values on the heatmap
    fmt=".2f",       # Format annotations to 2 decimal places
    cmap='viridis',  # Color map (dark blue for low, yellow for high)
    linewidths=.5,   # Lines between cells
    linecolor='white',
    square=True,
    vmin=0, vmax=1.0
)
plt.title("Cramér's V Heatmap: Redundancy Check on 9 Financial Features", fontsize=16)
plt.yticks(rotation=0, fontsize=10)
plt.xticks(rotation=90, fontsize=10)
plt.tight_layout()
plt.savefig('cramers_v_heatmap_9_features.png')
plt.close() # Close plot to prevent showing in environment if not needed

# --- 6. Output Statistical Table ---
print("\n--- Statistical Matrix (Cramér's V) for 9 Features ---")
print("High values (V >= 0.75) indicate strong redundancy between the two features.")
print(v_matrix_original.round(2))

# **FINANCE RELATED COLUMN CHOOSEN BASED ON CRAMER's V**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.cloud import bigquery
from scipy.stats import chi2_contingency

# --- Configuration ---
PROJECT_ID = 'stgw-shared-ai-dev'
DATASET_ID = 'parquet_data_set'
SAMPLE_TABLE = 'IMP_profile_FE_SAMPLE'

# **CRITICAL SAFETY MEASURE: Sample Size**
# To prevent memory issues/crashes due to high cardinality, we sample the data.
# Start with a small number (e.g., 50,000) and increase if your runtime can handle it.
N_SAMPLE = 50000

# Initialize BigQuery Client
client = bigquery.Client(project=PROJECT_ID)
print(f"BigQuery Client initialized for project: {PROJECT_ID}")

# --- 1. Define the 9 Features to Analyze ---
# This list is designed to test for redundancy between Kept (3) and Dropped (6) features.
features_to_analyze = [
    # 3 Kept Features
    'mortgage_loan_imputed',
    'credit_tier_imputed',
    'card_type_imputed',
    # 6 Dropped/Redundant Candidates
    'mortgage_imputed',
    'credit_cards_imputed',
    'discretionary_spend_imputed',
    'wealth_rating_imputed',
    'credit_behavior_imputed',
    'personal_finance_profile_imputed'
]

column_list_str = ",\n    ".join(features_to_analyze)

# --- 2. Define Cramér's V Function ---
def cramers_v(x, y):
    """Calculates Cramér's V, a measure of association between two nominal variables."""
    # Ensure all values are strings to treat as categories
    x, y = x.astype(str), y.astype(str)

    confusion_matrix = pd.crosstab(x, y)
    if confusion_matrix.empty or confusion_matrix.shape[0] == 1 or confusion_matrix.shape[1] == 1:
        return 0.0

    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape

    phi2 = chi2 / n
    # Bias correction for categorical variables
    phi2_corrected = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    r_corrected = max(1, r - ((r-1)**2)/(n-1))
    k_corrected = max(1, k - ((k-1)**2)/(n-1))

    v = np.sqrt(phi2_corrected / min(r_corrected, k_corrected))
    return min(1.0, max(0.0, v))

# --- 3. Load Data from BigQuery (with Sampling) ---
print(f"Loading a random sample of {N_SAMPLE} rows for {len(features_to_analyze)} features from BigQuery table: {SAMPLE_TABLE}...")

# Use a two-stage process: query all data and sample in Pandas, OR sample directly in BigQuery
# For simplicity, we use the BigQuery sample function if available or a LIMIT:
sample_load_query = f"""
SELECT
    {column_list_str}
FROM
    `{PROJECT_ID}.{DATASET_ID}.{SAMPLE_TABLE}`
WHERE
    rand() < {N_SAMPLE / 1300000.0} -- Approximate percentage based on 1.3M total rows
LIMIT {N_SAMPLE}
"""

# Fetch the data into a Pandas DataFrame
try:
    df_original = client.query(sample_load_query).to_dataframe()
    # Final check on sample size
    if len(df_original) < N_SAMPLE * 0.8:
        print(f"Warning: Loaded {len(df_original)} rows, less than target sample size.")
    else:
        print(f"Data loaded successfully with {len(df_original)} rows.")
except Exception as e:
    print(f"Error loading data from BigQuery: {e}")
    # Exit if data load fails
    print("Cannot proceed without data. Please check your BigQuery connection and table path.")
    raise

# --- 4. Calculate Cramér's V Matrix ---
print("Calculating Cramér's V Matrix from sampled data...")
v_matrix_original = pd.DataFrame(index=features_to_analyze,
                                 columns=features_to_analyze,
                                 dtype=float)

for i in range(len(features_to_analyze)):
    for j in range(i, len(features_to_analyze)):
        col1 = features_to_analyze[i]
        col2 = features_to_analyze[j]

        v = cramers_v(df_original[col1], df_original[col2])
        v_matrix_original.loc[col1, col2] = v
        v_matrix_original.loc[col2, col1] = v # Symmetric matrix


# --- 5. Generate and Save Heatmap ---
plt.figure(figsize=(10, 8)) # Smaller size for 9 features
sns.heatmap(
    v_matrix_original.astype(float),
    annot=True,
    fmt=".2f",
    cmap='viridis',
    linewidths=.5,
    linecolor='white',
    square=True,
    vmin=0, vmax=1.0
)
plt.title("Cramér's V Heatmap: Redundancy Check on 9 Financial Features", fontsize=12)
plt.yticks(rotation=0, fontsize=9)
plt.xticks(rotation=90, fontsize=9)
plt.tight_layout()
plt.savefig('cramers_v_heatmap_9_features_final.png')
plt.close()

# --- 6. Output Statistical Table ---
print("\ncramers_v_heatmap_9_features_final.png saved.")
print("\n--- Statistical Matrix (Cramér's V) for 9 Features ---")
print("High values (V >= 0.75) indicate strong redundancy (yellow cells).")
print(v_matrix_original.round(2))

# **Check on 9 HNI-Type Behavior Features**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.cloud import bigquery
from scipy.stats import chi2_contingency

# --- Configuration ---
PROJECT_ID = 'stgw-shared-ai-dev'
DATASET_ID = 'parquet_data_set'
SAMPLE_TABLE = 'IMP_profile_FE_SAMPLE'
N_SAMPLE = 50000
TOTAL_ROWS_EST = 1300000

# Initialize BigQuery Client
client = bigquery.Client(project=PROJECT_ID)
print(f"BigQuery Client initialized for project: {PROJECT_ID}")

# --- 1. Define the 9 Features to Analyze (HNI-Type Behavior) ---
features_to_analyze = [
    # 4 Kept Features
    'travel_sites_imputed',
    'mobile_service_provider_imputed',
    'movie_genre_saw_in_theater_imputed',
    'number_of_vehicles_in_household_imputed',
    # 5 Highly Correlated Candidates to Drop
    'travel_imputed',
    'hotels_for_any_travel_imputed',
    'mobile_devices_imputed',
    'entertainment_sites_imputed',
    'auto_imputed'
]

column_list_str = ",\n    ".join(features_to_analyze)

# --- 2. Define Cramér's V Function ---
def cramers_v(x, y):
    """Calculates Cramér's V with bias correction for association between two nominal variables."""
    x, y = x.astype(str), y.astype(str)

    confusion_matrix = pd.crosstab(x, y)
    if confusion_matrix.empty or confusion_matrix.shape[0] <= 1 or confusion_matrix.shape[1] <= 1:
        return 0.0

    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape

    phi2 = chi2 / n
    phi2_corrected = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    r_corrected = max(1, r - ((r-1)**2)/(n-1))
    k_corrected = max(1, k - ((k-1)**2)/(n-1))

    v = np.sqrt(phi2_corrected / min(r_corrected, k_corrected))
    return min(1.0, max(0.0, v))

# --- 3. Load Data from BigQuery (with Sampling) ---
print(f"Loading a random sample of {N_SAMPLE} rows for {len(features_to_analyze)} features from BigQuery table: {SAMPLE_TABLE}...")

# Estimate the required sampling fraction
sample_rate = N_SAMPLE / TOTAL_ROWS_EST
if sample_rate > 1: sample_rate = 1.0

sample_load_query = f"""
SELECT
    {column_list_str}
FROM
    `{PROJECT_ID}.{DATASET_ID}.{SAMPLE_TABLE}`
WHERE
    rand() < {sample_rate}
LIMIT {N_SAMPLE}
"""

# Fetch the data into a Pandas DataFrame
try:
    df_original = client.query(sample_load_query).to_dataframe()
    print(f"Data loaded successfully with {len(df_original)} rows.")
except Exception as e:
    print(f"Error loading data from BigQuery: {e}")
    print("Cannot proceed without data. Please check your BigQuery connection and table path.")
    raise

# --- 4. Calculate Cramér's V Matrix ---
print("Calculating Cramér's V Matrix from sampled data...")
v_matrix_original = pd.DataFrame(index=features_to_analyze,
                                 columns=features_to_analyze,
                                 dtype=float)

for i in range(len(features_to_analyze)):
    for j in range(i, len(features_to_analyze)):
        col1 = features_to_analyze[i]
        col2 = features_to_analyze[j]

        v = cramers_v(df_original[col1], df_original[col2])
        v_matrix_original.loc[col1, col2] = v
        v_matrix_original.loc[col2, col1] = v # Symmetric matrix


# --- 5. Generate and Save Heatmap ---
FILE_NAME = 'cramers_v_heatmap_9_HNI_behavior.png'
plt.figure(figsize=(10, 8))
sns.heatmap(
    v_matrix_original.astype(float),
    annot=True,
    fmt=".2f",
    cmap='viridis',
    linewidths=.5,
    linecolor='white',
    square=True,
    vmin=0, vmax=1.0
)
plt.title("Cramér's V Heatmap: Redundancy Check on 9 HNI-Type Behavior Features", fontsize=12)
plt.yticks(rotation=0, fontsize=9)
plt.xticks(rotation=90, fontsize=9)
plt.tight_layout()
plt.savefig(FILE_NAME)
plt.close()

# --- 6. Output Statistical Table ---
print(f"\n{FILE_NAME} saved.")
print("\n--- Statistical Matrix (Cramér's V) for 9 HNI-Type Behavior Features ---")
print("High values (V >= 0.75) indicate strong redundancy (yellow cells).")
print(v_matrix_original.round(2))

# **# Finding Zero Correlation**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.cloud import bigquery
from scipy.stats import chi2_contingency

# --- Configuration ---
PROJECT_ID = 'stgw-shared-ai-dev'
DATASET_ID = 'parquet_data_set'
SAMPLE_TABLE = 'IMP_profile_FE_SAMPLE'
N_SAMPLE = 50000
TOTAL_ROWS_EST = 1300000

# Initialize BigQuery Client
client = bigquery.Client(project=PROJECT_ID)
print(f"BigQuery Client initialized for project: {PROJECT_ID}")

# --- 1. Define the 16 Features to Analyze (Core & Financial) ---
features_to_analyze = [
    'mortgage_loan_imputed',
    'credit_cards_used_most_imputed',
    'auto_loan_imputed',
    'Unlock_gender_imputed',
    'credit_tier_imputed',
    'family_income_decile_imputed',
    'average_household_income_imputed',
    'financial_beliefs_imputed',
    'loan_imputed',
    'credit_card_spend_imputed',
    'credit_card_balance_imputed',
    'equity_balance_imputed',
    'family_level_of_investable_assets_imputed',
    'net_worth_imputed',
     'discretionary_spend_imputed',
    'wealth_rating_imputed',
    'credit_cards_used_30days_imputed',
    'card_type_imputed'
]

column_list_str = ",\n    ".join(features_to_analyze)

# --- 2. Define Cramér's V Function ---
def cramers_v(x, y):
    """Calculates Cramér's V with bias correction for association between two nominal variables."""
    x, y = x.astype(str), y.astype(str)

    confusion_matrix = pd.crosstab(x, y)
    if confusion_matrix.empty or confusion_matrix.shape[0] <= 1 or confusion_matrix.shape[1] <= 1:
        return 0.0

    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape

    phi2 = chi2 / n
    phi2_corrected = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    r_corrected = max(1, r - ((r-1)**2)/(n-1))
    k_corrected = max(1, k - ((k-1)**2)/(n-1))

    v = np.sqrt(phi2_corrected / min(r_corrected, k_corrected))
    return min(1.0, max(0.0, v))

# --- 3. Load Data from BigQuery (with Sampling) ---
print(f"Loading a random sample of {N_SAMPLE} rows for {len(features_to_analyze)} features from BigQuery table: {SAMPLE_TABLE}...")

# Estimate the required sampling fraction
sample_rate = N_SAMPLE / TOTAL_ROWS_EST
if sample_rate > 1: sample_rate = 1.0

sample_load_query = f"""
SELECT
    {column_list_str}
FROM
    `{PROJECT_ID}.{DATASET_ID}.{SAMPLE_TABLE}`
WHERE
    rand() < {sample_rate}
LIMIT {N_SAMPLE}
"""

# Fetch the data into a Pandas DataFrame
try:
    df_original = client.query(sample_load_query).to_dataframe()
    print(f"Data loaded successfully with {len(df_original)} rows.")
except Exception as e:
    print(f"Error loading data from BigQuery: {e}")
    print("Cannot proceed without data. Please check your BigQuery connection and table path.")
    raise

# --- 4. Calculate Cramér's V Matrix ---
print("Calculating Cramér's V Matrix from sampled data...")
v_matrix_original = pd.DataFrame(index=features_to_analyze,
                                 columns=features_to_analyze,
                                 dtype=float)

for i in range(len(features_to_analyze)):
    for j in range(i, len(features_to_analyze)):
        col1 = features_to_analyze[i]
        col2 = features_to_analyze[j]

        v = cramers_v(df_original[col1], df_original[col2])
        v_matrix_original.loc[col1, col2] = v
        v_matrix_original.loc[col2, col1] = v # Symmetric matrix


# --- 5. Generate and Save Heatmap ---
FILE_NAME = 'cramers_v_heatmap_16_core_financial.png'
plt.figure(figsize=(14, 12))
sns.heatmap(
    v_matrix_original.astype(float),
    annot=True,
    fmt=".2f",
    cmap='viridis',
    linewidths=.5,
    linecolor='white',
    square=True,
    vmin=0, vmax=1.0
)
plt.title("Cramér's V Heatmap: Redundancy Check on 16 Core & Financial Features", fontsize=14)
plt.yticks(rotation=0, fontsize=9)
plt.xticks(rotation=90, fontsize=9)
plt.tight_layout()
plt.savefig(FILE_NAME)
plt.close()

# --- 6. Output Statistical Table ---
print(f"\n{FILE_NAME} saved.")
print("\n--- Statistical Matrix (Cramér's V) for 16 Core & Financial Features ---")
print("High values (V >= 0.75) indicate strong redundancy (yellow cells).")
print(v_matrix_original.round(2))

In [ ]:
dataset = "stgw-shared-ai-dev.parquet_data_set"
table = "IMP_profile_FE_SAMPLE"

columns = [
    "mortgage_loan_imputed",
    "credit_cards_used_most_imputed",
    "auto_imputed",
    "auto_loan_imputed",
    "Unlock_gender_imputed",
    "credit_tier_imputed",
    "family_income_decile_imputed",
    "average_household_income_imputed",
    "financial_beliefs_imputed",
    "loan_imputed",
    "credit_card_spend_imputed",
    "credit_card_balance_imputed",
    "equity_balance_imputed",
    "family_level_of_investable_assets_imputed",
    "net_worth_imputed",
    "investments_type_imputed",
    "credit_cards_used_30days_imputed",
    "card_type_imputed",
    "tech_learning_imputed",
    "streaming_system_device_use_imputed",
    "retail_sites_imputed",
    "batteries_imputed",
    "tv_brand_owned_imputed",
    "hh_cable_subscriptions_imputed",
    "cell_smartphones_tablets_imputed",
    "downloaded_streamed_from_internet_imputed",
    "camera_brand_owned_imputed",
    "news_sites_imputed",
    "technology_imputed",
    "residential_isp_imputed",
    "social_sites_imputed",
    "tv_when_acquired_imputed",
    "tablets_imputed",
    "sports_sites_imputed",
    "travel_sites_imputed",
    "mobile_service_provider_imputed",
    "radio_imputed",
    "i_device_types_imputed",
    "cable_sat_provider_imputed",
    "movie_genre_saw_in_theater_imputed",
    "number_of_vehicles_in_household_imputed",
    "automotive_insurance_imputed",
    "number_of_visits_imputed",
    "television_sites_imputed",
    "contributions_imputed",
    "grocery_stores_imputed",
    "grocery_soft_drinks_imputed",
    "grocery_hot_dogs_imputed",
    "grocery_crackers_imputed",
    "charitable_contributions_imputed"
]

case_statements = []
for col in columns:
    stmt = (
        "CASE "
        "WHEN {0} IS NULL OR TRIM({0}) = '' THEN '__MISSING__' "
        "WHEN {0} IN ("
        "SELECT {0} FROM `{1}.{2}` "
        "GROUP BY {0} "
        "ORDER BY COUNT(*) DESC "
        "LIMIT 50) THEN {0} "
        "ELSE 'OTHER' END AS {0}"
    ).format(col, dataset, table)
    case_statements.append(stmt)

# Combine into final SQL
select_clause = ",\n".join(case_statements)
sql = (
    f"CREATE OR REPLACE TABLE `{dataset}.IMP_profile_FE_SAMPLE_TOP50` AS\n"
    f"SELECT extern_tuid,\n{select_clause}\n"
    f"FROM `{dataset}.{table}`;"
)

print(sql)
